<a href="https://colab.research.google.com/github/yyduyuxuan/Machine-Learning-for-Data-Driven-Inventory-Replenishment-Evidence-from-the-M5-Retail-Dataset/blob/main/Baseline_Adaptive_Item_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting - Baseline

In [ ]:
!pip install statsforecast[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.6 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Demand type sampling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
sales_path = '/content/drive/MyDrive/Colab Notebooks/Supervised Project/sales_long_val_5y.parquet'
adi_path = '/content/drive/MyDrive/Colab Notebooks/Supervised Project/adi_cv2_df.parquet'


sales_long_val = pd.read_parquet(sales_path)
adi_cv2_df = pd.read_parquet(adi_path)
display(sales_long_val.head())
display(adi_cv2_df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.12/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,enc_std_store_id_dept_id,enc_mean_item_id,enc_std_item_id,enc_mean_item_id_state_id,enc_std_item_id_state_id,enc_mean_item_id_store_id,enc_std_item_id_store_id,ROP_L3,ROP_L7,ROP_L14
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3,2011-01-29,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,NaN,NaN,NaN
1,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,0,2011-01-30,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,NaN,NaN,NaN
2,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,2011-01-31,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,5.0,NaN,NaN
3,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,4,1,2011-02-01,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,7.0,NaN,NaN
4,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,5,4,2011-02-02,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,6.0,NaN,NaN


,series_id,ADI,CV2,Demand_Type,log_ADI,log_CV2
0,FOODS_1_001_CA_1,2.291018,0.555704,Lumpy,0.828997,-0.587518
1,FOODS_1_001_CA_2,1.934277,0.715303,Lumpy,0.659734,-0.335047
2,FOODS_1_001_CA_3,2.227008,1.157275,Lumpy,0.800659,0.146069
3,FOODS_1_001_CA_4,3.872470,0.313532,Intermittent,1.353893,-1.159851
4,FOODS_1_001_TX_1,3.065705,0.993451,Lumpy,1.120278,-0.006570


In [ ]:
target_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/target_uids.csv")

In [ ]:
import pandas as pd
import numpy as np

# target_df check
targets = target_df['unique_id'].astype(str).tolist()
print(f"[INFO] target unique_id count = {len(targets)} (expect 159)")

# unique_id
df_tmp = sales_long_val.copy()
df_tmp['unique_id'] = df_tmp['item_id'].astype(str) + '_' + df_tmp['store_id'].astype(str)

df_long = (
    df_tmp[df_tmp['unique_id'].isin(targets)]
      .loc[:, ['unique_id','item_id','store_id','state_id','dept_id','d','date','sales']]
      .sort_values(['unique_id','d'])
      .reset_index(drop=True)
)

print(f"[INFO] df_long rows = {len(df_long):,}  | series covered = {df_long['unique_id'].nunique()}")

# Check missing series
found = set(df_long['unique_id'].unique())
missing = [u for u in targets if u not in found]
if missing:
    print(f"[WARN] missing series in df_long: {len(missing)}")
    print(missing[:10])
else:
    print("[OK] all target series found in df_long.")

# Check demand type
type_map = (
    adi_cv2_df.rename(columns={'series_id': 'unique_id'})
             .loc[:, ['unique_id','ADI','CV2','Demand_Type']]
)


type_159 = type_map[type_map['unique_id'].isin(targets)].copy()

# type statistics
type_count = (
    type_159.groupby('Demand_Type', dropna=False)
            .size()
            .reset_index(name='n_series')
            .sort_values('n_series', ascending=False)
            .reset_index(drop=True)
)


display(type_count)


df_long = df_long.merge(type_159, on='unique_id', how='left')

display(df_long.head())

[INFO] target unique_id count = 159 (expect 159)
[INFO] df_long rows = 270,266  | series covered = 159
[OK] all target series found in df_long.


,Demand_Type,n_series
0,Intermittent,68
1,Lumpy,47
2,Smooth,32
3,Erratic,12


,unique_id,item_id,store_id,state_id,dept_id,d,date,sales,ADI,CV2,Demand_Type
0,FOODS_1_145_CA_1,FOODS_1_145,CA_1,CA,FOODS_1,883,2013-06-29,0,2.412358,0.45293,Intermittent
1,FOODS_1_145_CA_1,FOODS_1_145,CA_1,CA,FOODS_1,884,2013-06-30,0,2.412358,0.45293,Intermittent
2,FOODS_1_145_CA_1,FOODS_1_145,CA_1,CA,FOODS_1,885,2013-07-01,0,2.412358,0.45293,Intermittent
3,FOODS_1_145_CA_1,FOODS_1_145,CA_1,CA,FOODS_1,886,2013-07-02,3,2.412358,0.45293,Intermittent
4,FOODS_1_145_CA_1,FOODS_1_145,CA_1,CA,FOODS_1,887,2013-07-03,2,2.412358,0.45293,Intermittent


In [ ]:
df_long.to_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/df_long_1ytest.csv')

In [ ]:
import pandas as pd
import numpy as np

targets = set(target_df['unique_id'])

df_ts = (
    df_long[df_long['unique_id'].isin(targets)]
      .rename(columns={'date':'ds', 'sales':'y'})
      [['ds','unique_id','y']]
      .assign(ds=lambda x: pd.to_datetime(x['ds']),
              y =lambda x: pd.to_numeric(x['y'], errors='coerce'))
      .drop_duplicates(['unique_id','ds'])
      .sort_values(['unique_id','ds'])
      .reset_index(drop=True)
)

print(f"[INFO] after filter: series={df_ts['unique_id'].nunique()} | rows={len(df_ts):,}")


FILL_MISSING = 'zero'
def _complete_daily(g):
    cal = pd.DataFrame({'ds': pd.date_range(g['ds'].min(), g['ds'].max(), freq='D')})
    gg  = cal.merge(g[['ds','y']], on='ds', how='left')
    if FILL_MISSING == 'zero':
        gg['y'] = gg['y'].fillna(0.0)
    gg['unique_id'] = g['unique_id'].iat[0]
    return gg

df_ts_full = (
    df_ts.groupby('unique_id', group_keys=False)
         .apply(_complete_daily)
         .reset_index(drop=True)[['ds','unique_id','y']]
)

print(f"[OK] ts_full: series={df_ts_full['unique_id'].nunique()} | rows={len(df_ts_full):,} | "
      f"missing%={df_ts_full['y'].isna().mean()*100:.2f}")

display(df_ts_full.head())

[INFO] after filter: series=159 | rows=270,266
[OK] ts_full: series=159 | rows=270,266 | missing%=0.00


/tmp/ipython-input-2736176550.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_complete_daily)


,ds,unique_id,y
0,2013-06-29,FOODS_1_145_CA_1,0
1,2013-06-30,FOODS_1_145_CA_1,0
2,2013-07-01,FOODS_1_145_CA_1,0
3,2013-07-02,FOODS_1_145_CA_1,3
4,2013-07-03,FOODS_1_145_CA_1,2


In [ ]:
df_ts_full.to_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/df_ts_full_1ytest.csv')

# Baseline forecast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_ts_full = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/df_ts_full_1ytest.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_long = df_ts_full.copy()
df_long.head()

,Unnamed: 0,ds,unique_id,y
0,0,2013-06-29,FOODS_1_145_CA_1,0
1,1,2013-06-30,FOODS_1_145_CA_1,0
2,2,2013-07-01,FOODS_1_145_CA_1,0
3,3,2013-07-02,FOODS_1_145_CA_1,3
4,4,2013-07-03,FOODS_1_145_CA_1,2


In [ ]:
import pandas as pd

# Windows same as ML
def make_windows():
    return {
        'train_start': pd.Timestamp('2011-10-29'),
        'train_end'  : pd.Timestamp('2014-10-28'),
        'val_start'  : pd.Timestamp('2014-10-29'),
        'val_end'    : pd.Timestamp('2015-01-28'),
        'test_start' : pd.Timestamp('2015-01-29'),
        'test_end'   : pd.Timestamp('2016-01-28'),
    }

def split_windows(df: pd.DataFrame):
    need = {'unique_id','ds','y'}
    missing = need - set(df.columns)
    if missing:
        raise ValueError(f"df missing: {missing}")
    df = df.copy()
    df['ds'] = pd.to_datetime(df['ds'])

    W = make_windows()
    train_df = df[(df['ds'] >= W['train_start']) & (df['ds'] <= W['train_end'])].copy()
    val_df   = df[(df['ds'] >= W['val_start'])   & (df['ds'] <= W['val_end'])].copy()
    test_df  = df[(df['ds'] >= W['test_start'])  & (df['ds'] <= W['test_end'])].copy()

    return train_df, val_df, test_df

train_df, val_df, test_df = split_windows(df_long)
print(train_df.shape, val_df.shape, test_df.shape)

(154363, 4) (14628, 4) (58035, 4)


In [ ]:
train_df, val_df, test_df = split_windows(df_long)

print("TRAIN  :", train_df['ds'].min().date(), "→", train_df['ds'].max().date(), "| rows:", train_df.shape[0])
print("VAL    :", val_df['ds'].min().date(),   "→", val_df['ds'].max().date(),   "| rows:", val_df.shape[0])
print("TEST   :", test_df['ds'].min().date(),  "→", test_df['ds'].max().date(),  "| rows:", test_df.shape[0])

TRAIN  : 2011-10-29 → 2014-10-28 | rows: 154363
VAL    : 2014-10-29 → 2015-01-28 | rows: 14628
TEST   : 2015-01-29 → 2016-01-28 | rows: 58035


In [ ]:
def split_unique_id(df):
    # unique_id: {dept}_{item}_{state}_{store}
    parts = df['unique_id'].str.split('_', expand=True)

    df = df.copy()
    df['dept_id']  = parts[0] + "_" + parts[1]        # FOODS_1
    df['item_id']  = parts[0] + "_" + parts[1] + "_" + parts[2]  # FOODS_1_145
    df['state_id'] = parts[3]                         # CA / TX / WI
    df['store_id'] = parts[4]                         # 1 / 2 / 3 / 4

    return df

df_long = split_unique_id(df_long)

print(df_long[['unique_id','dept_id','item_id','state_id','store_id']].head())
print("Total stores:", df_long['state_id'].nunique(), df_long['store_id'].nunique())
print("Distinct store combos:", df_long[['state_id','store_id']].drop_duplicates().values.tolist())

          unique_id  dept_id      item_id state_id store_id
0  FOODS_1_145_CA_1  FOODS_1  FOODS_1_145       CA        1
1  FOODS_1_145_CA_1  FOODS_1  FOODS_1_145       CA        1
2  FOODS_1_145_CA_1  FOODS_1  FOODS_1_145       CA        1
3  FOODS_1_145_CA_1  FOODS_1  FOODS_1_145       CA        1
4  FOODS_1_145_CA_1  FOODS_1  FOODS_1_145       CA        1
Total stores: 3 4
Distinct store combos: [['CA', '1'], ['CA', '3'], ['CA', '4'], ['TX', '1'], ['TX', '2'], ['TX', '3'], ['WI', '1'], ['WI', '3']]


In [ ]:
# 8 STORES
store_groups = dict(tuple(df_long.groupby(['state_id','store_id'])))

for (st, sid), sub_df in store_groups.items():
    n_items = sub_df['unique_id'].nunique()
    n_rows  = len(sub_df)
    print(f"Store {st}_{sid}: {n_items} items, {n_rows} rows")

Store CA_1: 20 items, 34452 rows
Store CA_3: 20 items, 33857 rows
Store CA_4: 20 items, 34221 rows
Store TX_1: 19 items, 32448 rows
Store TX_2: 20 items, 34802 rows
Store TX_3: 20 items, 34480 rows
Store WI_1: 20 items, 32527 rows
Store WI_3: 20 items, 33479 rows


In [ ]:
# Baseline Forecast
import os
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import Tuple, Optional
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA, Theta, SeasonalNaive, CrostonClassic, HoltWinters
)

# Model
OUT_DIR = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

MODELS = [
    AutoARIMA(),
    Theta(),
    SeasonalNaive(season_length=7),
    CrostonClassic()
]

# Predict function: Given a time series (index=ds), fit a single model and forecast H steps ahead, returning a Series of forecasts indexed by ds.
def sf_fit_predict(y_train: pd.Series, H: int, model) -> pd.Series:
    sdf = pd.DataFrame({'unique_id': 's', 'ds': y_train.index, 'y': y_train.values})
    sf = StatsForecast(models=[model], freq='D', n_jobs=1)
    sf.fit(sdf)
    fcst = sf.predict(h=H)
    col = model.__class__.__name__
    return fcst.set_index('ds')[col]

def run_one_group(df_long_group: pd.DataFrame, W: dict, refit_every: int = 7):
    df = (df_long_group[['unique_id', 'ds', 'y']]
          .dropna(subset=['y'])
          .copy())
    if not np.issubdtype(df['ds'].dtype, np.datetime64):
        df['ds'] = pd.to_datetime(df['ds'])
    df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)

    # true values
    val_truth  = df[(df['ds'] >= W['val_start'])  & (df['ds'] <= W['val_end'])][['unique_id','ds','y']].rename(columns={'y':'y_true'})
    test_truth = df[(df['ds'] >= W['test_start']) & (df['ds'] <= W['test_end'])][['unique_id','ds','y']].rename(columns={'y':'y_true'})

    def can_use_arima(y):
        return (len(y) >= 12) and (y.nunique() > 1) and np.isfinite(y).all()

    def can_use_theta(y):
        return (len(y) >= 10) and (y.nunique() > 1)

    def is_intermittent(y, thr=0.6):
        nz_ratio = (y != 0).mean()
        return nz_ratio < thr

    def can_use_croston(y):
        return is_intermittent(y) and (y.sum() > 0)


    val_rows, test_rows_static, test_rows_adapt = [], [], []

    for uid, g in df.groupby('unique_id', sort=False):
        s = g.set_index('ds')['y'].astype(float)

        y_tr  = s.loc[W['train_start']:W['train_end']]
        y_val = s.loc[W['val_start']  :W['val_end']]
        y_tst = s.loc[W['test_start'] :W['test_end']]

        H_val, H_test = len(y_val), len(y_tst)
        if H_val == 0 or H_test == 0 or len(y_tr) == 0:
            continue

        # choose models
        models_to_try = []
        if can_use_arima(y_tr):  models_to_try.append(AutoARIMA())
        if can_use_theta(y_tr):  models_to_try.append(Theta())
        models_to_try.append(SeasonalNaive(season_length=7))
        if can_use_croston(y_tr): models_to_try.append(CrostonClassic())

        for m in models_to_try:
            name = m.__class__.__name__

            # val (train -> val)
            try:
                vhat = sf_fit_predict(y_tr, H_val, m).reindex(y_val.index).clip(lower=0)
                val_rows.append(pd.DataFrame({'unique_id': uid, 'ds': y_val.index, 'model': name, 'y_hat': vhat.values}))
            except Exception:
                continue

            # test (train+val -> test)
            try:
                that = sf_fit_predict(pd.concat([y_tr, y_val]), H_test, m).reindex(y_tst.index).clip(lower=0)
                test_rows_static.append(pd.DataFrame({'unique_id': uid, 'ds': y_tst.index, 'model': name, 'y_hat': that.values}))
            except Exception:
                pass

        # Adaptive by every 7 days
        history = s.loc[:W['val_end']].copy()
        test_idx = pd.date_range(W['test_start'], W['test_end'], freq='D')

        for m in models_to_try:
            name = m.__class__.__name__
            preds = []
            h = history.copy()

            for i in range(0, len(test_idx), refit_every):
                horizon    = min(refit_every, len(test_idx) - i)
                days_chunk = test_idx[i : i + horizon]

                try:
                    fc = sf_fit_predict(h, horizon, m)
                except Exception:
                    preds = []
                    break

                for j, t in enumerate(days_chunk):
                    p = float(fc.iloc[j]) if j < len(fc) else 0.0
                    preds.append({'unique_id': uid, 'ds': t, 'model': name, 'y_hat': max(0.0, p)})

                # update observations
                for t in days_chunk:
                    if t in s.index:
                        h = pd.concat([h, pd.Series(s.loc[t], index=[t])])

            if preds:
                test_rows_adapt.append(pd.DataFrame(preds))

    # merge with true values
    val_daily         = pd.concat(val_rows,         ignore_index=True) if val_rows         else pd.DataFrame()
    test_daily_static = pd.concat(test_rows_static, ignore_index=True) if test_rows_static else pd.DataFrame()
    test_daily_adapt  = pd.concat(test_rows_adapt,  ignore_index=True) if test_rows_adapt  else pd.DataFrame()

    if not val_daily.empty:
        val_daily = val_daily.merge(val_truth,  on=['unique_id','ds'], how='left')
    if not test_daily_static.empty:
        test_daily_static = test_daily_static.merge(test_truth, on=['unique_id','ds'], how='left')
    if not test_daily_adapt.empty:
        test_daily_adapt  = test_daily_adapt.merge(test_truth,  on=['unique_id','ds'], how='left')

    return val_daily, test_daily_static, test_daily_adapt

def save_three(df_val, df_ts, df_ta, tag: str):
    out_val = os.path.join(OUT_DIR, f"{tag}_val_daily.parquet")
    out_ts  = os.path.join(OUT_DIR, f"{tag}_test_daily_static.parquet")
    out_ta  = os.path.join(OUT_DIR, f"{tag}_test_daily_adapt.parquet")
    if df_val is not None and not df_val.empty: df_val.to_parquet(out_val, index=False)
    if df_ts  is not None and not df_ts.empty:  df_ts.to_parquet(out_ts,  index=False)
    if df_ta  is not None and not df_ta.empty:  df_ta.to_parquet(out_ta,  index=False)
    return out_val, out_ts, out_ta


In [ ]:
W = make_windows()

In [ ]:
import gc
from tqdm.auto import tqdm
import pandas as pd

REFIT_EVERY = 7

print(f"[RUN] Baseline forecasting by store … (refit_every={REFIT_EVERY})")
for (st, sid), sub_df in tqdm(store_groups.items(), total=len(store_groups)):
    tag = f"{st}_{sid}"
    if not pd.api.types.is_datetime64_any_dtype(sub_df['ds']):
        sub_df = sub_df.copy()
        sub_df['ds'] = pd.to_datetime(sub_df['ds'])

    print(f"\n===> Store {tag}: series={sub_df['unique_id'].nunique()} | rows={len(sub_df):,}")

    try:
        val_df, test_static_df, test_adapt_df = run_one_group(
            df_long_group=sub_df,
            W=W,
            refit_every=REFIT_EVERY
        )

        v_path, s_path, a_path = save_three(val_df, test_static_df, test_adapt_df, tag)
        print(f"[DONE {tag}] "
              f"val={val_df.shape if val_df is not None else (0,0)} "
              f"test_static={test_static_df.shape if test_static_df is not None else (0,0)} "
              f"test_adapt={test_adapt_df.shape if test_adapt_df is not None else (0,0)}")
        print("   saved to:\n   ", v_path, "\n   ", s_path, "\n   ", a_path)

    except Exception as e:
        print(f"[ERROR {tag}] {e}")

    finally:
        try: del val_df
        except NameError: pass
        try: del test_static_df
        except NameError: pass
        try: del test_adapt_df
        except NameError: pass
        gc.collect()

[RUN] Baseline forecasting by store … (refit_every=7)


  0%|          | 0/8 [00:00<?, ?it/s]


===> Store CA_1: series=20 | rows=34,452
[DONE CA_1] val=(6256, 5) test_static=(24820, 5) test_adapt=(24820, 5)
   saved to:
    /content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/CA_1_val_daily.parquet 
    /content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/CA_1_test_daily_static.parquet 
    /content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/CA_1_test_daily_adapt.parquet

===> Store CA_3: series=20 | rows=33,857
[DONE CA_3] val=(5888, 5) test_static=(23360, 5) test_adapt=(23360, 5)
   saved to:
    /content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/CA_3_val_daily.parquet 
    /content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/CA_3_test_daily_static.parquet 
    /content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/CA_3_test_daily_adapt.parquet

===> Store CA_4: series=20 | rows=34,221
[DONE CA_4] val=(6624, 5) test_static=(26280, 5) test_

In [ ]:
import os, glob
from pathlib import Path
import pandas as pd
import numpy as np

OUT_DIR = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

def concat_by_pattern(pattern: str) -> pd.DataFrame:
    files = sorted(glob.glob(os.path.join(OUT_DIR, pattern)))
    parts = []
    for fp in files:
        try:
            df = pd.read_parquet(fp)
        except Exception as e:
            print(f"[WARN] skip {fp} -> {e}")
            continue

        # structure
        need = {'unique_id','ds','model','y_hat','y_true'}
        for c in (need - set(df.columns)):
            df[c] = np.nan
        df = df[['unique_id','ds','model','y_hat','y_true']].copy()

        # data type check
        df['ds']     = pd.to_datetime(df['ds'])
        df['model']  = df['model'].astype(str)
        df['y_hat']  = pd.to_numeric(df['y_hat'], errors='coerce')
        df['y_true'] = pd.to_numeric(df['y_true'], errors='coerce')

        stem = Path(fp).stem
        store_tag = stem.split("_val_daily")[0].split("_test_daily_static")[0].split("_test_daily_adapt")[0]
        df['store_tag'] = store_tag

        parts.append(df)

    if parts:
        out = pd.concat(parts, ignore_index=True)
        out = out.sort_values(['unique_id','ds','model'], kind='mergesort').reset_index(drop=True)
    else:
        out = pd.DataFrame(columns=['unique_id','ds','model','y_hat','y_true','store_tag'])
    return out

# Merge
val_daily_all         = concat_by_pattern("*_val_daily.parquet")
test_daily_static_all = concat_by_pattern("*_test_daily_static.parquet")
test_daily_adapt_all  = concat_by_pattern("*_test_daily_adapt.parquet")

print("val_daily_all:",         val_daily_all.shape)
print("test_daily_static_all:", test_daily_static_all.shape)
print("test_daily_adapt_all:",  test_daily_adapt_all.shape)

# Save
val_path_all   = os.path.join(OUT_DIR, "ALL_val_daily.parquet")
tstat_path_all = os.path.join(OUT_DIR, "ALL_test_daily_static.parquet")
tadpt_path_all = os.path.join(OUT_DIR, "ALL_test_daily_adapt.parquet")

val_daily_all.to_parquet(val_path_all, index=False)
test_daily_static_all.to_parquet(tstat_path_all, index=False)
test_daily_adapt_all.to_parquet(tadpt_path_all, index=False)

print("\n[SAVED]")
print(val_path_all)
print(tstat_path_all)
print(tadpt_path_all)


val_daily_all: (49680, 6)
test_daily_static_all: (197100, 6)
test_daily_adapt_all: (197100, 6)

[SAVED]
/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_val_daily.parquet
/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_daily_static.parquet
/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_daily_adapt.parquet


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
val_daily_all = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_val_daily.parquet')
test_daily_adapt_all = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_daily_adapt.parquet')
display(val_daily_all.head())
display(test_daily_adapt_all.head())

,unique_id,ds,model,y_hat,y_true,store_tag
0,FOODS_1_145_CA_1,2014-10-29,AutoARIMA,1.429112,3,CA_1
1,FOODS_1_145_CA_1,2014-10-29,SeasonalNaive,0.000000,3,CA_1
2,FOODS_1_145_CA_1,2014-10-29,Theta,1.321453,3,CA_1
3,FOODS_1_145_CA_1,2014-10-30,AutoARIMA,1.429112,3,CA_1
4,FOODS_1_145_CA_1,2014-10-30,SeasonalNaive,0.000000,3,CA_1


,unique_id,ds,model,y_hat,y_true,store_tag
0,FOODS_1_145_CA_1,2015-01-29,AutoARIMA,1.746914,5,CA_1
1,FOODS_1_145_CA_1,2015-01-29,SeasonalNaive,2.000000,5,CA_1
2,FOODS_1_145_CA_1,2015-01-29,Theta,1.782293,5,CA_1
3,FOODS_1_145_CA_1,2015-01-30,AutoARIMA,1.746914,4,CA_1
4,FOODS_1_145_CA_1,2015-01-30,SeasonalNaive,0.000000,4,CA_1


In [ ]:
test_daily_static_all = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_daily_static.parquet')
display(test_daily_static_all.head())

,unique_id,ds,model,y_hat,y_true,store_tag
0,FOODS_1_145_CA_1,2015-01-29,AutoARIMA,1.746914,5,CA_1
1,FOODS_1_145_CA_1,2015-01-29,SeasonalNaive,2.000000,5,CA_1
2,FOODS_1_145_CA_1,2015-01-29,Theta,1.782293,5,CA_1
3,FOODS_1_145_CA_1,2015-01-30,AutoARIMA,1.746914,4,CA_1
4,FOODS_1_145_CA_1,2015-01-30,SeasonalNaive,0.000000,4,CA_1


# CrostonClassic

In [ ]:
from statsforecast.models import CrostonClassic
import pandas as pd
from tqdm.auto import tqdm

def sf_fit_predict(y_train: pd.Series, H: int, model) -> pd.Series:
    sdf = pd.DataFrame({'unique_id': 's', 'ds': y_train.index, 'y': y_train.values})
    sf = StatsForecast(models=[model], freq='D', n_jobs=1)
    sf.fit(sdf)
    fcst = sf.predict(h=H)
    col = model.__class__.__name__
    return fcst.set_index('ds')[col]

In [ ]:
croston_uids_done = set()

if 'model' in val_daily_all.columns:
    croston_uids_done |= set(val_daily_all.loc[val_daily_all['model']=='CrostonClassic','unique_id'].unique())

if 'model' in test_daily_static_all.columns:
    croston_uids_done |= set(test_daily_static_all.loc[test_daily_static_all['model']=='CrostonClassic','unique_id'].unique())

if 'model' in test_daily_adapt_all.columns:
    croston_uids_done |= set(test_daily_adapt_all.loc[test_daily_adapt_all['model']=='CrostonClassic','unique_id'].unique())

all_uids = set(df_long['unique_id'].unique())
uids_need_croston = sorted(all_uids - croston_uids_done)

print(f"[INFO] total series: {len(all_uids)}")
print(f"[INFO] series missing CrostonClassic: {len(uids_need_croston)}")

[INFO] total series: 159
[INFO] series missing CrostonClassic: 96


In [ ]:
def coerce_ds_y(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # ds -> datetime64
    if not np.issubdtype(out['ds'].dtype, np.datetime64):
        out['ds'] = pd.to_datetime(out['ds'], errors='coerce')
    # y -> numerical
    out['y'] = pd.to_numeric(out['y'], errors='coerce')
    out = out.dropna(subset=['ds', 'y']).sort_values(['unique_id', 'ds']).reset_index(drop=True)
    return out

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import CrostonClassic

def run_croston_only(df_in: pd.DataFrame, W: dict, refit_every: int = 7):

    df = coerce_ds_y(df_in)

    val_rows, test_rows_static, test_rows_adapt = [], [], []

    for uid, g in df.groupby('unique_id', sort=False):
        s = g.set_index('ds')['y'].astype(float)

        m_tr   = (s.index >= W['train_start']) & (s.index <= W['train_end'])
        m_val  = (s.index >= W['val_start'])   & (s.index <= W['val_end'])
        m_test = (s.index >= W['test_start'])  & (s.index <= W['test_end'])

        y_tr, y_val, y_tst = s[m_tr], s[m_val], s[m_test]
        H_val, H_test = len(y_val), len(y_tst)
        if H_val == 0 or H_test == 0 or len(y_tr) == 0:
            continue

        # train -> val
        sf = StatsForecast(models=[CrostonClassic()], freq='D', n_jobs=1)
        sdf_tr = pd.DataFrame({'unique_id': 's', 'ds': y_tr.index, 'y': y_tr.values})
        sf.fit(sdf_tr)
        fc_val = sf.predict(h=H_val).set_index('ds')['CrostonClassic'].reindex(y_val.index)
        val_rows.append(pd.DataFrame({
            'unique_id': uid, 'ds': y_val.index, 'model': 'CrostonClassic', 'y_hat': fc_val.values
        }))

        # train+val -> test
        sdf_trv = pd.DataFrame({'unique_id': 's',
                                'ds': pd.Index(y_tr.index.tolist() + y_val.index.tolist()),
                                'y':  np.r_[y_tr.values, y_val.values]})
        sf.fit(sdf_trv)
        fc_test_static = sf.predict(h=H_test).set_index('ds')['CrostonClassic'].reindex(y_tst.index)
        test_rows_static.append(pd.DataFrame({
            'unique_id': uid, 'ds': y_tst.index, 'model': 'CrostonClassic', 'y_hat': fc_test_static.values
        }))

        # adaptive
        history = s[s.index <= W['val_end']].copy()
        test_idx = pd.date_range(W['test_start'], W['test_end'], freq='D')
        preds = []
        h = history.copy()
        for i in range(0, len(test_idx), refit_every):
            horizon = min(refit_every, len(test_idx) - i)
            days_chunk = test_idx[i:i+horizon]

            sdf_h = pd.DataFrame({'unique_id': 's', 'ds': h.index, 'y': h.values})
            sf.fit(sdf_h)
            fc = sf.predict(h=horizon).set_index('ds')['CrostonClassic']

            for j, t in enumerate(days_chunk):
                p = float(fc.iloc[j]) if j < len(fc) else 0.0
                preds.append({'unique_id': uid, 'ds': t, 'model': 'CrostonClassic', 'y_hat': max(0.0, p)})


            for t in days_chunk:
                if t in s.index:
                    h = pd.concat([h, pd.Series(s.loc[t], index=[t])])

        test_rows_adapt.append(pd.DataFrame(preds))

    # merge true
    val_truth  = df[(df['ds'] >= W['val_start'])  & (df['ds'] <= W['val_end'])][['unique_id','ds','y']].rename(columns={'y':'y_true'})
    test_truth = df[(df['ds'] >= W['test_start']) & (df['ds'] <= W['test_end'])][['unique_id','ds','y']].rename(columns={'y':'y_true'})

    val_daily         = pd.concat(val_rows,         ignore_index=True) if val_rows         else pd.DataFrame()
    test_daily_static = pd.concat(test_rows_static, ignore_index=True) if test_rows_static else pd.DataFrame()
    test_daily_adapt  = pd.concat(test_rows_adapt,  ignore_index=True) if test_rows_adapt  else pd.DataFrame()

    if not val_daily.empty:
        val_daily = val_daily.merge(val_truth,  on=['unique_id','ds'], how='left')
    if not test_daily_static.empty:
        test_daily_static = test_daily_static.merge(test_truth, on=['unique_id','ds'], how='left')
    if not test_daily_adapt.empty:
        test_daily_adapt  = test_daily_adapt.merge(test_truth,  on=['unique_id','ds'], how='left')

    return val_daily, test_daily_static, test_daily_adapt

In [ ]:
val_c_extra_parts, test_s_c_extra_parts, test_a_c_extra_parts = [], [], []
print("[RUN] Croston-only for missing series …")
for (st, sid), sub_df in tqdm(store_groups.items(), total=len(store_groups)):
    sub_need = sub_df[sub_df['unique_id'].isin(uids_need_croston)]
    if sub_need.empty:
        continue

    # ds -> datetime64, y -> numeric
    sub_need = coerce_ds_y(sub_need)

    v_new, ts_new, ta_new = run_croston_only(sub_need, W, refit_every=7)

    if not v_new.empty:
        v_new['store_tag'] = f"{st}_{sid}"
        val_c_extra_parts.append(v_new)
    if not ts_new.empty:
        ts_new['store_tag'] = f"{st}_{sid}"
        test_s_c_extra_parts.append(ts_new)
    if not ta_new.empty:
        ta_new['store_tag'] = f"{st}_{sid}"
        test_a_c_extra_parts.append(ta_new)


# merge all resutls
val_c_extra  = pd.concat(val_c_extra_parts,  ignore_index=True) if val_c_extra_parts  else pd.DataFrame()
test_s_c_extra= pd.concat(test_s_c_extra_parts,ignore_index=True) if test_s_c_extra_parts else pd.DataFrame()
test_a_c_extra= pd.concat(test_a_c_extra_parts,ignore_index=True) if test_a_c_extra_parts else pd.DataFrame()

print("Croston补充：val",  val_c_extra.shape,
      "| test_static", test_s_c_extra.shape,
      "| test_adapt", test_a_c_extra.shape)

# whole table
val_daily_all          = pd.concat([val_daily_all,          val_c_extra],  ignore_index=True)
test_daily_static_all  = pd.concat([test_daily_static_all,  test_s_c_extra],ignore_index=True)
test_daily_adapt_all   = pd.concat([test_daily_adapt_all,   test_a_c_extra],ignore_index=True)

[RUN] Croston-only for missing series …


  0%|          | 0/8 [00:00<?, ?it/s]

Croston补充：val (8832, 6) | test_static (35040, 6) | test_adapt (35040, 6)


In [ ]:
test_daily_adapt_all.head(20)

,unique_id,ds,model,y_hat,y_true,store_tag
0,FOODS_1_145_CA_1,2015-01-29,AutoARIMA,1.746914,5,CA_1
1,FOODS_1_145_CA_1,2015-01-29,SeasonalNaive,2.000000,5,CA_1
2,FOODS_1_145_CA_1,2015-01-29,Theta,1.782293,5,CA_1
3,FOODS_1_145_CA_1,2015-01-30,AutoARIMA,1.746914,4,CA_1
4,FOODS_1_145_CA_1,2015-01-30,SeasonalNaive,0.000000,4,CA_1
5,FOODS_1_145_CA_1,2015-01-30,Theta,1.781170,4,CA_1
6,FOODS_1_145_CA_1,2015-01-31,AutoARIMA,1.746914,2,CA_1
7,FOODS_1_145_CA_1,2015-01-31,SeasonalNaive,5.000000,2,CA_1
8,FOODS_1_145_CA_1,2015-01-31,Theta,1.780047,2,CA_1
9,FOODS_1_145_CA_1,2015-02-01,AutoARIMA,1.746914,2,CA_1


In [ ]:
val_daily_all.to_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_val_daily_withcroston.parquet', index=False)
test_daily_static_all.to_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_daily_static_withcroston.parquet', index=False)
test_daily_adapt_all.to_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_daily_adapt_withcroston.parquet', index=False)

# Sum demands in 3L

Build a table of L-day forward demand sums from daily predictions and truths.

    Expects columns: ['unique_id','ds','model','y_hat','y_true'].
    Returns: ['unique_id','ds','model','forecast_sum_L{L}','true_sum_L{L}'].

In [ ]:
import pandas as pd

def make_lead_sum_table(daily_df: pd.DataFrame, L: int = 3) -> pd.DataFrame:

    req = {'unique_id','ds','model','y_hat','y_true'}
    missing = req - set(daily_df.columns)
    if missing:
        raise ValueError(f"Input is missing columns: {missing}")

    df = daily_df[['unique_id','ds','model','y_hat','y_true']].copy()
    if not pd.api.types.is_datetime64_any_dtype(df['ds']):
        df['ds'] = pd.to_datetime(df['ds'])

    df = df.sort_values(['unique_id','model','ds'])

    # Forward L-day sum aligned to day t: sum over t..t+L-1
    df[f'forecast_sum_L{L}'] = (
        df.groupby(['unique_id','model'])['y_hat']
          .transform(lambda s: s.shift(-(L-1)).rolling(L).sum())
    )
    df[f'true_sum_L{L}'] = (
        df.groupby(['unique_id','model'])['y_true']
          .transform(lambda s: s.shift(-(L-1)).rolling(L).sum())
    )

    out = df[['unique_id','ds','model', f'forecast_sum_L{L}', f'true_sum_L{L}']].copy()
    # Drop the last L-1 rows per series/model that don’t have a full forward window
    out = out.dropna(subset=[f'forecast_sum_L{L}', f'true_sum_L{L}'])
    return out

In [ ]:
# Validation window
val_sum_L3   = make_lead_sum_table(val_daily_all, L=3)

# Test (static)
test_static_L3 = make_lead_sum_table(test_daily_static_all, L=3)

# Test (adaptive)
test_adapt_L3  = make_lead_sum_table(test_daily_adapt_all, L=3)

In [ ]:
test_adapt_L3.tail(10)

,unique_id,ds,model,forecast_sum_L3,true_sum_L3
197055,HOUSEHOLD_2_371_WI_3,2016-01-17,Theta,1.635376,0.0
197059,HOUSEHOLD_2_371_WI_3,2016-01-18,Theta,1.635057,2.0
197063,HOUSEHOLD_2_371_WI_3,2016-01-19,Theta,1.780631,2.0
197067,HOUSEHOLD_2_371_WI_3,2016-01-20,Theta,1.926207,2.0
197071,HOUSEHOLD_2_371_WI_3,2016-01-21,Theta,2.071787,0.0
197075,HOUSEHOLD_2_371_WI_3,2016-01-22,Theta,2.071476,0.0
197079,HOUSEHOLD_2_371_WI_3,2016-01-23,Theta,2.071165,0.0
197083,HOUSEHOLD_2_371_WI_3,2016-01-24,Theta,2.070855,0.0
197087,HOUSEHOLD_2_371_WI_3,2016-01-25,Theta,2.070544,0.0
197091,HOUSEHOLD_2_371_WI_3,2016-01-26,Theta,1.710047,2.0


In [ ]:
val_sum_L3.to_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/val_sum_L3_1y.parquet', index=False)
test_static_L3.to_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/test_static_L3_1y.parquet', index=False)
test_adapt_L3.to_parquet('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/test_adapt_L3_1y.parquet', index=False)

# Error Evaluation

In [ ]:
import numpy as np
import pandas as pd

# d -> numeric
def _ensure_dnum_col(df):
    if 'd' in df.columns and pd.api.types.is_numeric_dtype(df['d']):
        return df['d'].astype(int)
    s = df['d'].astype(str).str.extract(r'(\d+)', expand=False)
    return pd.to_numeric(s, errors='coerce')

def build_train_forward_sum(full_history: pd.DataFrame, L: int, train_end_d: int) -> pd.DataFrame:
    req = {'item_id','store_id','d','sales'}
    miss = req - set(full_history.columns)
    if miss:
        raise ValueError(f"Missing columns in full_history: {miss}")

    df = full_history[['item_id','store_id','d','sales']].copy()
    df['d_num'] = _ensure_dnum_col(df)
    df = df[df['d_num'].notna()].sort_values(['item_id','store_id','d_num'])

    df['train_sum_L'] = (
        df.groupby(['item_id','store_id'])['sales']
          .transform(lambda s: s.shift(-(L-1)).rolling(L).sum())
    )
    last_t = int(train_end_d) - (L - 1)
    df = df[(df['d_num'] <= last_t)].dropna(subset=['train_sum_L'])

    return df[['item_id','store_id','d_num','train_sum_L']]

# unique_id
def parse_unique_id(unique_id: pd.Series) -> pd.DataFrame:
    sp = unique_id.str.split('_', expand=True)
    out = pd.DataFrame({
        'unique_id': unique_id.values,
        'item_id':   sp[[0,1,2]].agg('_'.join, axis=1),
        'store_id':  sp[[3,4]].agg('_'.join, axis=1),
    })
    return out.drop_duplicates('unique_id')

# RMSSE denominator
def build_denom_once(full_history: pd.DataFrame, L: int, train_end_d: int, all_unique_ids: pd.Series) -> pd.DataFrame:
    uid_map = parse_unique_id(all_unique_ids.drop_duplicates())
    train_sums = build_train_forward_sum(full_history, L=L, train_end_d=train_end_d)
    train_sums_uid = train_sums.merge(uid_map, on=['item_id','store_id'], how='inner')

    def _denom_from_series(s):
        v = s.to_numpy()
        if v.size < 2:
            return np.nan
        d = np.diff(v)
        return float(np.mean(d*d))

    denom_df = (
        train_sums_uid
        .sort_values(['unique_id','d_num'])
        .groupby('unique_id', observed=True)['train_sum_L']
        .agg(_denom_from_series)
        .rename('denom')
        .reset_index()
    )
    return denom_df

In [ ]:
def evaluate_forward_sum_errors_fast(
    agg_df: pd.DataFrame,     # ['unique_id','ds','model', forecast_sum_L{L}, true_sum_L{L}]
    denom_df: pd.DataFrame,   # ['unique_id','denom']
    L: int
) -> pd.DataFrame:
    fcol = f'forecast_sum_L{L}'
    tcol = f'true_sum_L{L}'

    need = {'unique_id','ds','model',fcol,tcol}
    miss = need - set(agg_df.columns)
    if miss:
        raise ValueError(f"agg_df is missing: {miss}")

    df = agg_df[['unique_id','model',fcol,tcol]].copy()

    # Vectorized residual/squared error
    df['res'] = df[fcol] - df[tcol]
    df['se']  = df['res'] * df['res']

    # Calculate RMSE / ME / mean μ for each group at once
    g = (
        df.groupby(['unique_id','model'], observed=True)
          .agg(RMSE=('se',  lambda x: np.sqrt(np.mean(x))),
               ME  =('res', np.mean),
               MU  =(tcol,  np.mean))
          .reset_index()
    )
    g['Norm_RMSE'] = g['RMSE'] / (g['MU'] + 1e-8)
    g['Norm_ME']   = g['ME']   / (g['MU'] + 1e-8)

    # RMSSE
    g = g.merge(denom_df, on='unique_id', how='left')
    g['RMSSE'] = np.where(g['denom'] > 0, g['RMSE'] / np.sqrt(g['denom']), np.nan)
    g['note']  = np.where(g['denom'] > 0, None, 'no/short train series or denom=0')

    out = g.rename(columns={'unique_id':'SeriesName','model':'Model'})[
        ['SeriesName','Model','RMSE','Norm_RMSE','ME','Norm_ME','RMSSE','note']
    ]
    return out

In [ ]:
results_dir = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/results_state_dept_3y_train/"

In [ ]:
# Read train_end_d
with open(os.path.join(results_dir, "cv_meta.pkl"), "rb") as f:
    meta = pickle.load(f)
train_end_d = meta['train_end_d']

all_uids = pd.concat([
    val_sum_L3['unique_id'],
    test_static_L3['unique_id'],
    test_adapt_L3['unique_id'],
], ignore_index=True)

denom_df = build_denom_once(
    full_history=sales_long_val,
    L=3,
    train_end_d=train_end_d,
    all_unique_ids=all_uids
)

# Results
err_val    = evaluate_forward_sum_errors_fast(val_sum_L3,     denom_df, L=3)
err_static = evaluate_forward_sum_errors_fast(test_static_L3, denom_df, L=3)
err_adapt  = evaluate_forward_sum_errors_fast(test_adapt_L3,  denom_df, L=3)

display(err_adapt.sort_values(['SeriesName','Model']))

/tmp/ipython-input-845567594.py:23: FutureWarning: The provided callable <function mean at 0x793cdc3e6700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(RMSE=('se',  lambda x: np.sqrt(np.mean(x))),
/tmp/ipython-input-845567594.py:23: FutureWarning: The provided callable <function mean at 0x793cdc3e6700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(RMSE=('se',  lambda x: np.sqrt(np.mean(x))),
/tmp/ipython-input-845567594.py:23: FutureWarning: The provided callable <function mean at 0x793cdc3e6700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(RMSE=('se',  lambda x: np.sqrt(np.mean(x))),


,SeriesName,Model,RMSE,Norm_RMSE,ME,Norm_ME,RMSSE,note
0,FOODS_1_145_CA_1,AutoARIMA,3.610356,0.655933,0.098940,0.017976,1.313037,None
1,FOODS_1_145_CA_1,CrostonClassic,3.783855,0.687454,0.198284,0.036024,1.376137,None
2,FOODS_1_145_CA_1,SeasonalNaive,5.056468,0.918664,0.083102,0.015098,1.838969,None
3,FOODS_1_145_CA_1,Theta,3.753326,0.681908,0.000654,0.000119,1.365034,None
4,FOODS_1_145_CA_3,AutoARIMA,3.500059,0.654674,0.079180,0.014810,1.038242,None
...,...,...,...,...,...,...,...,...
631,HOUSEHOLD_2_371_WI_1,Theta,3.177584,0.705913,-0.031281,-0.006949,1.368793,None
632,HOUSEHOLD_2_371_WI_3,AutoARIMA,2.154947,1.105022,-0.075228,-0.038576,1.129303,None
633,HOUSEHOLD_2_371_WI_3,CrostonClassic,2.158278,1.106731,-0.245127,-0.125697,1.131049,None
634,HOUSEHOLD_2_371_WI_3,SeasonalNaive,2.938898,1.507020,-0.011080,-0.005682,1.540134,None


In [ ]:
err_adapt.to_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/err_adapt.csv')
err_static.to_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/err_static.csv')
err_val.to_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/err_val.csv')